**Projektuppgift 1 Statistik**
Denna notebook kommer undersöka påverkan av olika faktorer för användning av kollektivtravik genom att använda regressionsanalys och datavisualisernig med data som delades i canvasrummet från främst Kommun- och landstingsdatabasen. Beskrivning av scatter plots, histogram och kod kommer ske direkt efter varje kodcell och figur. Syftet är att öva på datahantering och datavisualissering i Python samt att enkelt använda  regression för a göra en statistiskmodell utifrån data. 

**A) Datahantering och Beskrivande statistik**
*Vald indikator (N07900) Tillgång till bredband om minst 100 Mbit/s, andel (%)*
För den första delupggiften är den valda indinkatorn tillgång till snabbt bredband. Det kan verka som detta är en helt orelaterad varibel men hypotetiskt kan det finnas korrelation. Snabbare bredband gör det mer möjligt att arbeta hemifrån och därmed minskar behovet att pendla. Samtidigt är snabbare bredband snarare kopplat mellan stad gentemot landsbygd så om en koppling mellan faktorn och kollektivtrafikresande uppstår är det möjligt att detta är orsaken till eventuella resultat. Oavsett är det intressant att undersöka denna indikator, och i senare deluppgifter när man ska moddelera andelen kollektivtrafikresor kommer andra indikatorer väljas. 



In [13]:

import plotly.express as px;
import pandas as pd;

# Ange sökvägen till CSV-filen
csv_path = r"C:\Users\fabia\AH1023\Projekt\Projekt_1\pt_data.csv"

# Läs in datan från CSV-filen
df = pd.read_csv(csv_path);


Kodcellen ovan laddar först plotly express som kommer användas för att göra figurer i notebooken och pandas som används för datahantering samt ger dom korta namn för att enkelt kunna användas i funktioner. Sedan anges sökvägen till datafilen till upggiften och den läses in och ges variabelnamnet df.

In [14]:


col = "N07900"
s = pd.to_numeric(df[col], errors="coerce").dropna()
# ange/beräkna för den valda indikatorn: min, max, medel, median och standardavvikelse. 
print(f"{col} — count: {s.count()}")
print(f"min:    {s.min()}")
print(f"max:    {s.max()}")
print(f"mean:   {s.mean()}")
print(f"median: {s.median()}")
print(f"std:    {s.std()}")
# plotta histogram utifrån vald variabel 
fig = px.histogram(df, x="N07900", nbins=30, title="Histogram över tillgång på bredband om minst 100 Mbit/s (%)",labels={"N07900": "Andel (%)"}, template="simple_white")
fig.show()

N07900 — count: 290
min:    0.18836826
max:    96.01088065
mean:   35.08771286182758
median: 30.78244899
std:    21.884208603019868


Koden här väljer vi vår indikator (N07900) som är en kolumn i vår datafil och konverterar innehållet till numeriska tal. Sedan använder vi inbyggda funktioner för att räkna ut medel, median, max och minvärde samt standardavvikelse. Resultaten verkar i stort sätt rimlig även om minvärdet är mycket lågt men det är fortfarande tänkbart att det finns enstaka kommuner där tillgången till bredband är mycket låg.

Vi använder sedan plotly för att göra ett histogram med vår indikator vi kan i histogrammet se att de flesta kommuner har under 40% av invånarna tillgång till sanbbt bredband i hemmet och i mycket få kommuner är andelen över 70-80%. 

In [15]:
fig = px.histogram(df, x="N07900", nbins=30, title="Histogram över tillgång på bredband om minst 100 Mbit/s (%) uppdelat på kommungrupp",labels={"N07900": "Andel (%)"}, template="simple_white", facet_col="Kommungrupp", width=1200);
fig.show()

Här har vi använt Kommungrupp som är en till kolumn i datan för att dela in kommunerna i separata histogram beronde på sin kommungrupp. Det finns ett antal intressant aobservationer att ta fån detta, vi kan direkt se att Kommungrupp 1 (storstadskommunerna) har en mycket hög andel av befolkningen tillgång til snabbt Bredband. I kommungrupp 4 (    ), 7 () och 9 () kan vi se att större delen av kommunerna har en låg andel (under 50%) tillgång till snabbt bredband. Kommungrupp 2 () har jämnast spridning med kommuner med både hög och låg andel av befolkningen med snabbt bredband och allt däremellan. Att storstadskommunerna har störst andel av befolkningen med tillgång till snabbt bredband samt att andelen i Kommungrupp 7 är låg är inte överraskande. 

**B) Regressionsanalys**
Vi gör här en scatter plot där vi låtert variabeln pt_share (andelen i procent av resorna som sker med kollektivtrafik) vara Y och tillgången till bredband är X


In [16]:


#Gör en scatterplot där du plottar Y mot förklaringsvariabeln du valde i A. 
fig = px.scatter(df, x="N07900", y="pt_share", title="pt_share vs tillgång på bredband om minst 100 Mbit/s (%)", template="simple_white", labels={"N07900": "Andel (%)", "pt_share": "Andel med kollektivtrafik (%)"})
fig.show()

Utifrån scatter plotten är det mycket svårt att se något direkt mönster, i den övre änden av tillgång till snabbt bredband syns visserligen fler kommuner med hög andel kollektivtrafikresor men det finns många med låg andel kollektivtrafikresor också. Längst i övre högra hörnet med mycket hög andel med snabbt bredbrand och hög andel kollektivtrafikresor ser vi ett antal kommuner vilket verkar vara våra storstadskommuner. Redan här kan vi se att indakatorn förmodligen inte kommer vara användbar i en regresionsmodell för pt_share. 

In [17]:
#gör ett subset av datframen med de valda variablerna 

df_subset = df.filter(["pt_share", "U85001", "U85421", "N00914", "N07900", "U01803", "N02937", "Kommungrupp","N00905"], axis=1)
#scatter plot matrix av alla 8 valda variabler 
fig = px.scatter_matrix(df_subset, dimensions=["pt_share", "U85001", "U85421", "N00914", "N07900", "U01803", "N02937","N00905", "Kommungrupp"], title="Scatter plot matrix av 8 valda variabler", height =1000, width=1300)
fig.show()


Koden väljer ut åtta av de tillgängliga versionerna och gör en scatter plot matrix tillsammans med pt share. (mer om scatter plot matrix) utifrån detta kan vi titta på raden ptshare där x är ptshare och på samma sätt som den individuella pt share och N07900. Valda variabler som verkar ha ett samband med pt_share alltså att det verkar finnas en tydlig ökning/minsking i scatter plotten när det är plootat mot pt_share är: N02937 invånare per km^2, U85001 Nettokostnad kollektivtrafik, N00905 mediannettoinkomst. Dessa indikatorer verkar har ett samband och indikatorerna verkar också rimliga. Ökad befolkningstäthet bör ha en positiv effekt på kollektivtrafikresande då det möjligör fler att fångas upp av samma hållplats eller linje. Hög nettokostnad för kollektitrafik kan leda till att man väljer andra färdessätt och scatter plotten för N00905 verkar visa att kommuner med högre medianettoinkomst har högre kollektivtrafikanvänding, troligtvis kopplat till att dessa kommuner ofta är större städer eller förorter.

In [18]:

import statsmodels.formula.api as smf
model_defined = smf.ols(formula='pt_share ~ N02937 + U85001 + N00905+C(Kommungrupp)', data=df_subset)
model_trained = model_defined.fit()
print(model_trained.summary())


                            OLS Regression Results                            
Dep. Variable:               pt_share   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     30.38
Date:                Thu, 20 Nov 2025   Prob (F-statistic):           1.73e-41
Time:                        12:31:01   Log-Likelihood:                 619.48
No. Observations:                 288   AIC:                            -1215.
Df Residuals:                     276   BIC:                            -1171.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0456    

Koden här laddar in statsmodels som smf, nästa linje använder statsmodels funktion för att passa in en funktion med minstakvadratmetoden för att göra en linjär regressionsmodell och outputar en sammanfattning av modellen. I kollumnen såg våra "dummy variabler" har stora koefficienter med kommungrupp 1 ger en ökning i pt_share, men som vi redan konstaterat har storstadskommunerna alla mycket högt användande av kollektivtrafik för att de är just storstärder. Samtidigt har även resterande kommungrupper ett mycket stort standardfel jämfört med våra valda indikatorer. För våra valda indikatorer är koefficienterna små men också standardavvikelsen, små koefficenter är också rimligt eftersom pt_share är en andel mellan 0 och 1 jämfört med våra variabler som kan vara i tusental (befolkningstäthet) och hundratal (medianinkomst).  

**C) Validering**
Första steget i valideringen är att dela in datan i df in två delar estimation_data från 75% av den ursprungliga datan valt slumpmässigt och sedan ta det som blev över och skapa en annan datamängd estimation data. Som vi sedan kan använda för att predidktera värden med vår modell.

In [19]:
estimation_data = df.sample(frac=0.75)

validation_data = df.drop(estimation_data.index)

Med våra datamängder kan vi igen använda färdiga funktioner för att prediktera nya värden utifrån modellen, vi tar samma indikatorer vi valde i modellen och validation data med vår model_trained får vi en variabel pt_share_predicted. Efter det så ordnas den datan in i kolumner. De sista linjerna kod gör en ny scatterplot som jämnför predikterade värden med de obseverade pt_share. 

In [20]:
pt_share_predicted = model_trained.predict(validation_data[["N02937", "U85001", "N00905", "Kommungrupp"]])

pt_share_predicted = pt_share_predicted.to_frame()

pt_share_predicted.columns = ['pt_share_predicted']

observed_plus_predicted = validation_data.join(pt_share_predicted)
fig1 = px.scatter(observed_plus_predicted, x="pt_share", y="pt_share_predicted", title="Observerade vs Predikterade värden", labels={"pt_share": "Observerade värden", "pt_share_predicted": "Predikterade värden"}, trendline="ols", trendline_color_override="green", template="simple_white")
fig1.show()


I scatter plotten kan man se hur lika de predikterade och observerade värderna på ptshare är när punkter är under linjen betyder det att det predikterade värdet är lägre än det faktiska. Om punkten är över betyder det att de observerade är lägre. Modellen visar sig relativt dålig på att förutspå de kommuner med lägre kollektivtrafikandel men relativt bättre på de högre. Nogrannheten av modellen är alltså ojämn men inga av extremvärderna verkar vara helt orimliga. 

**Avslutning och sammanfatting**
Det här projektet var en intressant övning i att kunna visualisera datan på ett bra sätt och själv kunna hitta indikatorer/parametrar för att göra statistiska regressionsmodeller. Eftersom mycket av koden för att göra själva modellerna gavs i uppgiftsbeskrivningen var detta den största utmaningen. Detta ledde till att uppgiften till stor del handlade om att tolka och visualisera data. Största felkällan i detta var att den första och de åtta variablerna i scatter plot matrisen valdes godtyckligt, men syftet med uppgiften var ej att bygga den bästa modellen möjligt utan att öva med att öva i python. De valda indikatoreran kunde i alla fall skapa en hyfsad regressionsmodell som kunde prediktera åtminstonde en del av värden a bra.  